# Optimal design of experiments

When unlabeled training data is abundant but obtaining the proper labels is
expensive, you may want to select only a subset of the data to label and use for
training your model. Sometimes, random choice may be good enough, but if the
data is redundant or skewed towards certain categories, or when your budget is
very low, then this might give bad results. *Design of Experiments* is the
process of selecting a good subset of the available samples to query the label
of.


## Preamble

Let's first import what we need:

In [ ]:
from math import ceil
from os import environ

import ipywidgets as widgets
import numpy as np
import picos as pc
from plotly import graph_objects as go
from plotly.offline import init_notebook_mode
from plotly.subplots import make_subplots
from skimage.transform import downscale_local_mean

LAYOUT = dict(
    margin=dict(l=0, r=0, b=0, t=0, pad=0),
    paper_bgcolor="rgba(0,0,0,0)",
    plot_bgcolor="rgba(0,0,0,0)",
)
init_notebook_mode()

# Detect if we are running on Binder; then we need to scale down problem sizes.
BINDER = "BINDER_SERVICE_HOST" in environ
if BINDER:
    print("Running in a Binder instance. Problems will be scaled down.")

In this notebook we will work with the
[Fashion-MNIST](https://github.com/zalandoresearch/fashion-mnist) image data
set, stored in the attached `fashion_mnist.npz`. Below we define one function to
load the images into a data matrix of varying size and one to visualize such a
matrix.

In [ ]:
def data_matrix(name, images, *, random_subset=None, downscale=None):
    """Convert a collection of 2D images to a PICOS data matrix.

    The rows of the returned matrix are (a random subset of) the scaled down,
    vectorized, and normalized input images.

    :param images: A n×w×h tensor of n many w×h grayscale images.
    :returns: An n'×(wh) data matrix, for n' ≤ n as specified.
    """
    if random_subset:
        if random_subset > len(images):
            raise ValueError("Not enough data!")

        images = images[np.random.choice(range(len(images)), random_subset)]

    if downscale:
        images = np.array(
            [downscale_local_mean(img, downscale) for img in images]
        )

    vectorized = np.reshape(images, (len(images), -1))
    normalized = vectorized / np.linalg.norm(vectorized, axis=1)[:, np.newaxis]

    return pc.Constant(name, normalized)


def plot(A, cols=10):
    """Recover images from a PICOS data matrix and plot them."""
    A = A.np2d  # Convert to NumPy.
    n, d = A.shape
    s = int(d**0.5)
    assert s**2 == d
    rows = ceil(n / cols)
    fig = make_subplots(rows=rows, cols=cols)
    for i, a_i in enumerate(A):
        row, col = i // cols + 1, i % cols + 1
        gray_image = (a_i.reshape((s, s)) / np.max(a_i) * 255).astype(int)
        rgb_image = gray_image[..., np.newaxis].repeat(3, axis=2)
        fig.add_trace(go.Image(z=rgb_image), row=row, col=col)
    fig.update_xaxes(showticklabels=False)
    fig.update_yaxes(showticklabels=False)
    fig.update_layout(LAYOUT | dict(height=rows * 100))
    fig.show()


# Load training and testing images from the dataset (attached).
dataset = np.load("fashion_mnist.npz")
A_img = dataset["train_images"]
T_img = dataset["test_images"]

# Plot some (normalized) images.
A = data_matrix("A", A_img, random_subset=40)
plot(A)

## Problem 2: Sample selection

The data set that we loaded conveniently comes with both data points (the
images) and associated labels (the type of fashion item depicted). This is not
always the case — in practice we are confronted with an abundance of *unlabeled*
data, which can be used in unsupervised learning tasks such as clustering but
which is not suited for supervised tasks like classification. Producing the
missing labels may be expensive, time consuming, error-prone, or even dangerous
if it involves performing real world measurements or experiments. In such
situations, we would like a **hint which data is the most valuable to put a
label on**.

The theory of **Optimal Design** (of experiments) provides statistical tools to
tackle this problem that can often be implemented as convex programs.


### Statistical background

Under simplifying assumptions, we can give a definite answer to the question
which samples are the best. In Optimal Design, one typically assumes a linear
regression model of the form

$$
    y = A\theta + \epsilon
$$

where the rows $a_i$ of $A$ are unlabeled samples (describing parameters for
experiments that one could perform), the entries $\epsilon_i$ of the error
vector $\epsilon$ are uncorellated random variables with mean
$\mathbb{E}[\epsilon_i] = 0$ and variance $\mathbb{V}[\epsilon_i] = \sigma^2$,
and where the entries $y_i$ of $y$ are (random) measurements that depend on both
the experiment performed and on the measurement error $\epsilon_i$. Finally,
$\theta$ is the parameter of the model that one would like to estimate.

A well known estimator for the parameter $\theta$ given experiments $A$ and
measurements $y$ is the *ordinary least squares (OLS)* estimator

$$
    \theta_\mathrm{OLS} = (A^T A)^{-1} A^T y,
$$

which, given that $A$ has full column rank, is the unique optimum solution of
the least-squares problem

$$
\begin{align*}
    \text{minimize} ~&~ {\lVert A\theta - y \rVert}^2. \\
\end{align*}
$$

The popularity of $\theta_\mathrm{OLS}$ is theoretically well motivated: it
is the [*best linear unbiased estimator
(BLUE)*](https://en.wikipedia.org/wiki/Gauss%E2%80%93Markov_theorem) for the
above setting, meaning that it is the estimator with the smallest variance among
all estimators $\hat{\theta}$ that are unbiased (i.e., $\mathbb{E}[\hat{\theta}]
= \theta$).

When we speak of the *variance* of an estimator, we refer to the fact that the
estimator is a random variable defined in terms of noisy measurements. Its
covariance matrix is

$$
    \mathbb{V}[\theta_\mathrm{OLS}] = \Sigma_A = \sigma^2 {(A^T A)}^{-1}.
$$

Note that **the "smaller" the matrix $\Sigma_A$ is, the more certain we can be
that $\theta_\mathrm{OLS}$ is a good estimate of the true parameter
$\theta$** (assuming that the linear model is an accurate description of
reality). In Optimal Design, we thus want to design the matrix of experiments
$A$ such that $\Sigma_A$ becomes "small". The inverse of $\Sigma_A$ is the
**information matrix**

$$
    M_A = \Sigma_A^{-1} = \sigma^{-2} A^T A.
$$

Intuitively, a "large" matrix $M_A$ indicates that the data points stored in the
rows of $A$ are very informative for our model. While we will stay in the realm
of linear regression for this notebook, note that the information matrix can be
defined more generally for nonlinear models, although then it will depend also
on $\theta$.


### Setting

Above we have introduced the information matrix $M_A$ that describes the
information content of the data in $A$ for fitting a linear model. Now, we will
assume that $A$ has a large number of rows $a_i$ but we have only a small
experimentation budget for producing the associated labels $y_i$. If we denote
by $z_i$ the number of times that we perform the experiment $a_i$, then the
information matrix becomes

$$
    M_A(z) = \frac{1}{\sigma^2} \sum_{i = 1}^n z_i a_i^T a_i.
$$

The term $\sigma^{-2}$ is just a constant that we may disregard in the following.
The *exact* optimal design problem is then the combinatorial problem

$$
\begin{align*}
    \text{maximize} ~&~ \phi(M_A(z)) \\
    \text{subject to} ~&~ \mathbf{1}^T z \leq \beta, \\
    \text{where} ~&~ z \in \mathbb{Z}_{\geq 0}^n. \\
\end{align*}
$$

Here $\beta \in \mathbb{Z}_{\geq 0}$ is our experimentation budget and $\phi$ is
a concave function that turns the positive semidefinite matrix $M_A(z)$ into a
scalar that we can maximize (such as $\operatorname{tr}$, $\log\det$, or
$\lambda_{\min}$).

The exact optimal design problem is NP-hard in general. However, the problem
becomes convex and tractable if we drop the requirement that $z$ is integral. We
define the (relaxed) **optimal design problem** as:

$$
\begin{align*}
    \text{maximize} ~&~ \phi(M_A(w)) \tag{1 | $\phi$-optimal design} \\
    \text{subject to} ~&~ \mathbf{1}^T w = 1, \\
    \text{where} ~&~ w \in \mathbb{R}_{\geq 0}^n \\
\end{align*}
$$

Here, $w$ represents a probability distribution (or importance assignment) over
the rows of $A$. In machine learning terms, we can think of it as a vector of
**sample weights**. The entries of $w_i^*$ that are zero in an optimal solution
$w^*$ correspond to *inessential* samples: we have good reason to not spend much
effort on labeling them.

While we're at it, let's define a function to compute $M_A(w)$ from a data
matrix $A$ and a vector variable $w$, for later use:

In [ ]:
def information_matrix(A, w):
    """Compute the information matrix M(w) for data A and sample weights w.

    The constant factor of 1/σ² is omitted.

    :param A: An n×d PICOS matrix containing n many d-dimensional samples.
    :param w: A PICOS n-dimensional vector expression.
    :returns: The d×d information matrix M(w) = ∑ᵢ wᵢ aᵢᵀ aᵢ (aᵢ i-th row of A).

    This function is a fast replacement for:

    >>> pc.sum(w[i] * (A[i, :].T * A[i, :]) for i in range(n)).renamed("M(w)")
    """
    n, d = A.shape
    C = np.array([np.outer(vec, vec).ravel() for vec in A.np2d])
    assert C.shape == (n, d**2)
    return (w.T * C).reshaped((d, d)).renamed(f"M({w.string})")

Additionally, we want a function that plots the images in $A$ corresponding to
nonzero entries of $w$:

In [ ]:
def show_solution(A, w):
    """Display images corresponding to nonzero entries of sample weights w."""
    w = w if isinstance(w, np.ndarray) else w.np
    support = np.where(w > 1e-6)[0]
    print(f"\nDesign w: {len(support)}/{len(w)} samples selected.")
    plot(A[support, :])

### Selecting data for one category: Bayesian $c$-optimal design

Different experimental designs (read: data points to label and associated
training weights) are obtained for different choices of the scalarization
function $\phi$. The first design we want to study is the Bayesian $c$-optimal
design, for a **row** vector $c \in \mathbb{R}^d$ and a parameter $\lambda > 0$:

$$
   \phi_{c,\lambda}(X) = -c (X + \lambda I)^{-1} c^T
$$

This criterion lets us identify data that provides information **best suited for
estimating the label of the fixed data point $c$**. In our example, if $c$ is
the image of a shoe, then we can expect the support of an optimal solution $w$
to identify a variety of shoes in the dataset. The *Bayesian* in the name refers
to a setup with prior knowledge on the parameter $\theta$ and the scalar
$\lambda$ has a statistical interpretation in this context; see [Pilz
(1991)](#References) for a monograph on Bayesian design. Here, however, we will
use $\lambda$ merely as a knob to control the number of data points selected.

Since $X = M(w)$ is positive semidefinite by construction as a sum of outer
products, we have that $X + \lambda I$ is positive definite and so taking the
inverse is both well-defined and a convex function. It follows that
$\phi_{c,\lambda}(M(w))$ is a concave function of $w$, so *maximizing* it
results indeed in a convex problem. For a long time, only a reformulation as an
SDP was known, which can be difficult to solve in high dimensions. A more
performant reformulation as an SOCP is due to Sagnol ([2011](#References)). More
recently, a connection to the following regression problem was discovered
([Sagnol and Pauwels, 2019](#References)):

$$
\begin{align*}
   \text{minimize} ~&~
      \lVert A^T x - c^T \rVert^2 + \lambda \lVert x \rVert_1^2
      \tag{2 | squared lasso} \\
      \text{where} ~&~ x \in \mathbb{R}^n.
\end{align*}
$$

When $\lVert x \rVert_1^2$ is replaced with $\lVert x \rVert_1$, then (2)
becomes the well-known lasso (or $\ell_1$-regularized) regression problem, in
which one tries to estimate $x$ from $A$ and $c$ (assuming $A^Tx \approx c^T$)
while simultaneously nudging $x$ to be a sparse vector: the penalty $\lVert x
\rVert_1 = \sum_{i=1}^n |x_i|$ is large if $x$ has many nonzero entries. To
compute a $c$-optimal design, we will make use of the following result (see
[Sagnol and Pauwels (2019)](#References) or [Sagnol and Pronzato (t.
a.)](#References) for a proof):

Let $\phi = \phi_{c,\lambda}$. Then,

1. the optimal value of (2) equals the optimal value of (1) multiplied by
   $\lambda$ and,
2. if $x$ solves (2), then $w$ with $w_i = \frac{|x_i|}{\lVert x
   \rVert_1}$ solves (1).


---

<div class="alert alert-info">

   **Task 2.1:**
   
   1. Implement the $\phi$-optimal design problem for $\phi =
   \phi_{c,\lambda}$ using the connection to the squared lasso regression
   problem described above.
   2. Vary $\lambda$. What do you observe?

  <details>
  <summary style='display: list-item'>PICOS and NumPy syntax (vector norms)</summary>

  | on paper | in picos |
  | --- | --- |
  | ${\lVert x \rVert}_p$ | `pc.Norm(x, p)` |
  | ${\lVert x \rVert}_2 = \lVert x \rVert$ | `pc.Norm(x, 2)` or `abs(x)` |
  | $t^2$ | `t**2` |

  For recovering the solution $w$, NumPy's entry-wise functions are useful:

  | on paper | in numpy |
  | --- | --- |
  | $\begin{pmatrix} \|x_1\| & \cdots & \|x_n\| \end{pmatrix}$ | `abs(x)` |
  | ${\lVert x \rVert}_1$ | `sum(abs(x))` |

  Recall that you can access the value of a PICOS expression `x` as a NumPy array
  via `x.np`.

  </details>

  <details>
  <summary style='display: list-item'>Recommended hint</summary>

  - PICOS does not currently have a class to represent a squared $\ell_1$-norm.
    You will have to improvise with an auxiliary variable.

  </details>

  <details>
  <summary style='display: list-item'>Hint</summary>

  - Write ${\lVert x \rVert}_1^2$ as $\tau^2$ with ${\lVert x \rVert}_1 \leq
    \tau$.

  </details>

</div>

In [ ]:
# You can decrease NUM_DATA and/or increment DOWNSCALE if solution is too slow.
NUM_DATA = 30 if BINDER else 300
DOWNSCALE = 1

# Load the data matrix from the training set at random.
A = data_matrix("A", A_img, random_subset=NUM_DATA, downscale=DOWNSCALE)
n, d = A.shape

# Load one reference image from the test set at random.
c = data_matrix("c", T_img, random_subset=1, downscale=DOWNSCALE)

# Inspect the expressions and plot the reference image.
print(repr(A), repr(c), sep="\n")
plot(c)

You can rerun the above cell to get a new reference sample.

In [ ]:
# TODO: Implement and solve the problem.
P = pc.Problem("Bayesian c-optimality")
l = pc.Constant("λ", 0.5)


# TODO: Recover a solution for the design vector w.
# w =

# Show selected images.
show_solution(A, w)

If all went well, you should see a handful of pieces of clothing that are mostly
of the same kind as the reference piece.

### Selecting data for multiple categories: Bayesian L(inear)-optimal design

So far we can extract data points whose labels are informative for predicting
the label of *one* reference point. This may be useful when a certain category
is underrepresented in a preliminary dataset and we are looking for additional
data that is similar yet varied. On its own, though, a $c$-optimal design is not
useful for training a classifier.

An L-optimal (or $A_K$-optimal) design is the extension of a $c$-optimal design
to **multiple reference vectors**, stored as the rows of a matrix $K \in
\mathbb{R}^{r \times d}$:

$$
   \phi_{K,\lambda}(X) = -K (X + \lambda I)^{-1} K^T
$$

The intuition is similar as for $c$-optimality: We are looking for a design that
jointly supports the (linear) estimation of all points in $K$. Again, $\lambda$
encodes prior knowledge on $\theta$ and serves as a knob to control sparsity.

By an extension of the earlier result, the $L$-optimality problem is equivalent
to the following variant of the *group lasso* problem:

$$
\begin{align*}
   \text{minimize} ~&~
      {\lVert A^T X - K^T \rVert}^2 + \lambda {\lVert X^T \rVert}_{2,1}^2
      \tag{3 | squared group lasso} \\
      \text{where} ~&~ X \in \mathbb{R}^{n \times r}.
\end{align*}
$$

Here,

$$
\begin{align*}
   \lVert B \rVert_{p,q}
   =
   {\left(
      \sum_{j=1}^n
      {\left(
         \sum_{i=1}^m
         {\left(
            |B_{i,j}|^p
         \right)}
      \right)}^{\frac{q}{p}}
   \right)}^{\frac{1}{q}}
\end{align*}
$$

denotes the matrix $L_{p,q}$-norm of a matrix $B \in \mathbb{R}^{m \times n}$,
which is the vector $\ell_{q}$-norm of the vector $\ell_{p}$-norms of the
columns of $B$. In particular,

$$
\begin{align*}
   {\lVert X^T \rVert}_{2,1}^2
   =
   \sum_{i=1}^n \sqrt{\sum_{j=1}^r X_{i,j}^2}
\end{align*}
$$

denotes the sum of the Euclidean norms of the rows of $X$.

More precisely, the extended result states that for the L-optimality criterion
$\phi = \phi_{K,\lambda}$, we have that

1. the optimal value of (3) equals the optimal value of (1) multiplied by
   $\lambda$ and,
2. if $X$ solves (3), then $w$ with $w_i = \frac{\Vert x_i \rVert}{\lVert X
   \rVert_{1, 2}}$ solves (1), where $x_i$ is the $i$-th row of $X$.


---

<div class="alert alert-info">

   **Task 2.2:** Implement the $\phi$-optimal design problem for $\phi =
   \phi_{K,\lambda}$ using its squared group lasso formulation.

   <details>
   <summary style='display: list-item'>PICOS and NumPy syntax (matrix norms)</summary>

   | on paper | in picos | in numpy | note |
   | --- | --- | --- | --- |
   | ${\lVert A \rVert}_{p,q}$ | `pc.Norm(A, p, q)` | | see also the [docs for Norm](https://picos-api.gitlab.io/picos/api/picos.expressions.exp_norm.html#picos.expressions.exp_norm.Norm) |
   | $\begin{pmatrix} \lVert a_1 \rVert & \cdots & \lVert a_m \rVert \end{pmatrix}$  | | `np.linalg.norm(A, axis=1)` | $a_i$ is $i$-th row of $A$
   | ${\lVert A^T \rVert}_{2,1}$ | | `sum(np.linalg.norm(A, axis=1))` | |

   Again, NumPy entry-wise functions are useful for recovering a solution value
   for $w$.

   </details>

</div>

In [ ]:
if BINDER:
    NUM_REFERENCE = 2
    NUM_DATA = 15
    DOWNSCALE = 3
    np.random.seed(4)  # Hand-pick a seed that solves quickly.
else:
    NUM_REFERENCE = 2
    NUM_DATA = 150
    DOWNSCALE = 2

# Load the data.
A = data_matrix("A", A_img, random_subset=NUM_DATA, downscale=DOWNSCALE)
K = data_matrix("K", T_img, random_subset=NUM_REFERENCE, downscale=DOWNSCALE)
(n, d), r = A.shape, NUM_REFERENCE

print(repr(A), repr(K), sep="\n")
plot(K)

You may rerun until a fashionable outfit appears.

In [ ]:
# TODO: Implement and solve the problem.
P = pc.Problem("L-optimality")
l = pc.Constant("λ", 0.5)


# TODO: Recover the design vector w.
# w =

# Show selected images alongside the one encoded by c.
show_solution(A, w)

Ideally, you should obtain a small collection that spans all reference samples well.

### Unsupervised selection: E(igenvalue)-optimal design

The above sample selection was semi-supervised: we had provided hand-picked
reference samples to tell the procedure which type of training data we are
interested in. Next, we want to study scalarizations of the information matrix
that do not depend on such guidance.

We also want to visualize how the selection is performed. To this end, we need
low-dimensional data that we can plot. Let's use principal component analysis to
project the images into three dimensions (`A_full` will contain the
full-dimensional and `A` the projected data):

In [ ]:
if BINDER:
    NUM_DATA = 50
    np.random.seed(2)
else:
    NUM_DATA = 500

# Project data to three dimensions using PCA.
A_full = data_matrix("A_full", A_img, random_subset=NUM_DATA)  # Load the data.
s = int(A_full.shape[1] ** 0.5)  # Store image side length for plotting.
A_full_np = A_full.np2d  # Convert the data matrix to NumPy.
if BINDER:  # Binder struggles with eigenvalue computations.
    A_full_np = np.array([downscale_local_mean(x, 4) for x in A_full_np])
A_full_np -= np.mean(A_full_np, 0)  # Center it.
S = np.cov(A_full_np, rowvar=False)  # Compute its covariance matrix S.
v = np.linalg.eigh(S)[1]  # Compute the eigenvectors of S.
v = v[:, : -(3 + 1) : -1]  # Select the three first principal eigenvectors.
A_np = np.dot(A_full_np, v)  # Project the data onto the space spanned by them.
A = pc.Constant("A", A_np)  # Load the result again as a PICOS expression.

print(repr(A_full), repr(A), sep="\n")

Let's also plot the projected data. You can hover over points to display the
original image:

In [ ]:
def show_image(trace, points, state):
    global s, fw2
    index = points.point_inds[0]
    vector = A_full.np[index]
    gray_image = (vector.reshape((s, s)) / np.max(vector) * 255).astype(int)
    rgb_image = gray_image[..., np.newaxis].repeat(3, axis=2)
    fw2.data[0].z = rgb_image


x, y, z = A.np.T
trace = go.Scatter3d(x=x, y=y, z=z, mode="markers", marker=dict(size=3))
fw1 = go.FigureWidget(data=trace, layout=LAYOUT)
fw1.update_layout(scene=dict(aspectmode="data"))
fw2 = go.FigureWidget(data=go.Image(z=np.zeros((s, s, 3))))
fw2.update_xaxes(title="selected image")
fw1.data[0].on_hover(show_image)
widgets.HBox([fw1, fw2])

With the above lasso reformulations, the fact that we were trying to maximize a
scalar criterion function of the information matrix $M_A(w)$ was rather
implicit. This will now change as we dive into some of the classic criteria,
starting with **E-optimality**. Here, we aim to **maximize the smallest eigenvalue
of the information matrix**,

$$
   \phi_\text{E}(X) = \lambda_{\min}(X).
$$

For the linear regression setting described in the *Statistical background*
section, this is equivalent to minimizing the largest eigenvalue (aka the
spectral radius) of the estimator covariance matrix $\Sigma$.

Let us restate the problem we are looking to solve:

$$
\begin{align*}
    \text{maximize} ~&~ \lambda_{\min}(M_A(w)) \tag{4 | E-optimal design} \\
    \text{subject to} ~&~ \mathbf{1}^T w = 1 \\
    \text{where} ~&~ w \in \mathbb{R}_{\geq 0}^n,
\end{align*}
$$

for

$$
    M_A(w) = \sum_{i = 1}^n w_i a_i^T a_i.
$$


---

<div class="alert alert-info">

   **Task 2.3:** Implement problem (4) for the low-dimensional data matrix `A`.

   <details>
   <summary style='display: list-item'>PICOS syntax (matrix inequalities)</summary>

  | on paper | in picos |
  | --- | --- |
  | $A \preceq B$ | `A << B` |
  | $A \succeq B$ | `A >> B` |
  | $I_n$ | `pc.I(n)` |

   </details>

   <details>
   <summary style='display: list-item'>Hint</summary>

   - Recall from the introduction talk that for $X \in \mathbb{S}^n$, it is
     $\lambda_{\min}(X) \leq \tau \Longleftrightarrow X \succeq \tau I_n$.

   </details>
</div>

In [ ]:
# TODO: Define a variable w and the information matrix M(w) for A.
# NOTE: You can use the function information_matrix defined earlier.
n, d = A.shape
# w =
# M =

# TODO: Implement and solve the problem.
P = pc.Problem("E-optimality")


# Display the solution in terms of the full-dimensional data.
show_solution(A_full, w)

# Save the solution for later.
M_E = ~M  # The ~ operator converts the current value to a constant expression.

We expect a small number of items that span three or more distinct categories.

Should you stumble upon an eigenvalue problem again, there is a shortcut in PICOS:

<details>
<summary style='display: list-item'>Spoiler</summary>

For a symmetric or hermitian matrix $A$:

| on paper | in picos | note |
| --- | --- | --- |
| $\lambda_{\text{min}}(A)$ | `pc.lambda_min(A)` | is a concave function of $A$, so you can maximize it
| $\lambda_{\text{max}}(A)$ | `pc.lambda_max(A)` | is a convex function of $A$, so you can minimize it

</details>


#### Geometric interpretation & Duality

We will next shed some light on how the above selection is made using a duality
argument. To this end, consider the **Lagrange dual problem** of problem (4):

$$
\begin{align*}
    \text{minimize} ~&~ \nu \tag{5} \\
    \text{subject to}
        ~&~ \operatorname{tr}(\Lambda) = 1, \\
        ~&~ a_i \Lambda a_i^T \leq \nu & \forall i \in [n], \\
    \text{where}
        ~&~ \Lambda \succeq 0.
\end{align*}
$$

The dual problem has one constraint for each variable and one (non-negative)
variable for each (in-)equality constraint of the primal problem.

<details>
<summary style='display: list-item'>Details</summary>

- The dual variable $\nu \in \mathbb{R}$ corresponds to the primal constraint
  $\mathbf{1}^T w = 1$,
- the dual variable $\Lambda \in \mathbb{S}^d$ with $\Lambda \succeq 0$ corresponds to the linear
  matrix inequality that you introduced for the eigenvalue bound,
- the dual constraint $\operatorname{tr}(\Lambda) = 1$ corresponds to the primal
  auxiliary variable that you introduced for the epigraph reformulation, and
- the dual constraint $a_i \Lambda a_i^T \leq \nu$ corresponds to $w_i$ for every $i \in [n]$.

</details>

By **strong duality**, problems (4) and (5) have the same optimal value, that is
$\nu = \lambda_{\min}(M_A(w))$ for a primal optimal $w$ and dual optimal $\nu$
and $\Lambda$. Informally, this means that formulation (5) is another way of
thinking about problem (4).

Problem (5) is *almost* useful for better understanding the E-optimal design. We
need to rearrange it a bit. First, observe that $\nu \geq 0$ in any feasible
solution (since $\Lambda \succeq 0$) and further $\nu \neq 0$ when $A$ has full
column rank (since $\Lambda \neq 0$). Dividing all constraints by $\nu$ and
performing the change of variable $X = \frac{1}{\nu} \Lambda$ gives us the
equivalent dual problem

$$
\begin{align*}
    \text{minimize} ~&~ \nu \tag{5b} \\
    \text{subject to}
        ~&~ \operatorname{tr}(X) = \frac{1}{\nu}, \\
        ~&~ a_i X a_i^T \leq 1 & \forall i \in [n], \\
    \text{where}
        ~&~ X \succeq 0.
\end{align*}
$$

<details>
<summary style='display: list-item'>Alternative formulation</summary>

As $\nu$ only denotes the inverse of the trace that we want to minimize, you may
find the dual written in the literature (e.g. in Boyd and Vandenberghe
([2014, problem 7.30](#References))) more compactly as

$$
\begin{align*}
    \text{maximize} ~&~ \operatorname{tr}(X) \tag{5c} \\
    \text{subject to}
        ~&~ a_i X a_i^T \leq 1 & \forall i \in [n], \\
    \text{where}
        ~&~ X \succeq 0.
\end{align*}
$$

Observe that (5b) and (5c) have the same solution (value of $X$ at optimality)
but not the same optimal value, so (5c) is not strictly a dual of problem (4).
However, only (5c) is a convex problem because of the nonlinear equality
constraint in (5b).

</details>

We will next make use of the fact that the dual constraint $a_i X a_i^T \leq 1$
corresponds to the $i$-th entry of the primal variable $w$, which encodes the
E-optimal design. By the **complementary slackness** condition — another
statement relating primal and dual problem — we have for every $i \in [n]$ that

$$
    w_i (a_i X a_i^T - 1) = 0
$$

holds whenever $w$ and $(X, \nu)$ form an optimal solution pair for problems (4)
and (5b). In other words, **only those data points $a_i$ can be part of the
design (i.e., $w_i > 0$) for which $a_i X a_i^T = 1$ for an optimal dual
solution $X$.** Geometrically, this means that all selected $a_i$ lie on the
hull of the ellipsoid $\mathcal{E} = \{x \in \mathbb{R}^d \mid x^T X x \leq
1\}$!

The next task is to obtain an optimal dual solution $X$ for plotting the
ellipsoid $\mathcal{E}$. There are two options to do so: we could implement the
dual problem and solve it separately or we could make use of the fact that PICOS
provides us with a dual solution as a byproduct of solving the primal: the
optimal values of dual variables are stored in the `dual` properties of the
corresponding primal constraint objects.

---

<div class="alert alert-info">

   **Task 2.4 / Option 1:**

   1. Implement and solve the dual problem (5c) (see *Alternative formulation*) to obtain a dual optimal solution $X$.

   **Option 2:**

   1. Edit your implementation of problem (4) above: store the linear matrix inequality constraint in a Python variable `lmi` before adding it to the problem.
   2. Obtain a dual solution for $\Lambda$ in problem (5) from `lmi.dual`. Use it to compute a solution $X$ for problem (5b).


   <details>
   <summary style='display: list-item'>PICOS syntax for Option 1</summary>

   | on paper | in picos | note |
   | --- | --- | --- |
   | $X \in \mathbb{S}^d$ | `X = pc.SymmetricVariable("X", d)` | |
   | $A_{i,j}$ | `A[i,j]` | entry in $i$-th row and $j$-th column of $A$ |
   | $A_{i,\cdot} = a_i$ | `A[i,:]` | $i$-th row of $A$ as a row vector |
   | $A_{\cdot,j}$ | `A[:,j]` | $j$-th column of $A$ as a column vector |

   </details>

   <details>
   <summary style='display: list-item'>Adding constraint families</summary>

   You can write, e.g.,

   ```
   problem += constraint1, constraint2
   ```

   to add multiple constraints at once. You may also supply them as a list. In particular, you may use Python [list comprehensions](https://docs.python.org/3/tutorial/datastructures.html#list-comprehensions) to define a family of constraints (here $x_i^2 \leq y_i \; \forall i \in [n]$):

   ```
   problem += [x[i]**2 <= y[i] for i in range(n)]
   ```

   If you supply constraints as a list, PICOS tries to guess a compact string representation for the family. Try it out by printing the dual problem!

   </details>

   <details>
   <summary style='display: list-item'>Obtaining dual values</summary>

   Instead of, say,
   
   ```
   problem += A*x <= b 
   ```

   you can write

   ```
   constraint = A*x <= b 
   problem += constraint
   ```

   and, after solving `problem`, access the dual value associated with the constraint via

   ```
   x = constraint.dual
   ```

   Note that `dual` returns values as a CVXOPT matrix. These are normally interchangeable with NumPy matrices. In case of doubt, you can write:

   ```
   x = np.array(constraint.dual)
   ```

   </details>

</div>

In [ ]:
# TODO: Obtain an optimal dual solution X.



def unit_ball(*, res=20):
    """Generate points on the 3D unit sphere."""
    u, v = np.mgrid[0 : 2 * np.pi : res * 2j, 0 : np.pi : res * 1j]
    u, v = u.ravel(), v.ravel()
    return np.stack((np.cos(u) * np.sin(v), np.sin(u) * np.sin(v), np.cos(v)))


def ellipsoid(X, *, res=20):
    """Generate points on an ellipsoid given by a quadratic form."""
    L = np.linalg.cholesky(np.linalg.inv(X))
    return L @ unit_ball(res=res)


# Collect samples supported by w in a submatrix S of A.
support = np.where(w.np > 1e-6)[0]
S = A[support, :]

# Plot the ellipsoid given by the dual solution X.
x, y, z = A.np.T
sx, sy, sz = S.np.T
ex, ey, ez = ellipsoid(X)
trace1 = go.Scatter3d(x=x, y=y, z=z, name="data", mode="markers", marker_size=3)
trace2 = go.Scatter3d(
    x=sx, y=sy, z=sz, name="selected", mode="markers", marker_size=7
)
kwargs = dict(alphahull=0, opacity=0.3, showlegend=True, hoverinfo="skip")
trace3 = go.Mesh3d(x=ex, y=ey, z=ez, name="tight dual constraint", **kwargs)
go.Figure(
    data=[trace1, trace2, trace3],
    layout=dict(
        title_text="E-optimal design visualized",
        margin=dict(t=50, b=0, l=0, r=0),
        height=600,
    ),
)

The figure should explain why in three dimensions we get only a couple of
categories and why possibly some of them are represented by multiple samples.
Also worth noting: optimal designs are not robust to outliers. Imagine what
would happen to the ellipsoid if there was a point far outside the cloud!

(If $X$ has eigenvalues close to zero, then the ellipsoid will be extremely
stretched. If this happens, re-running the notebook should give you another
instance that's more pleasant to look at.)

### Bonus: A(verage)-optimal design

Another classic is the **A-optimal** design, which maximizes the criterion

$$
   \phi_{\text{A}}(X) = -\operatorname{tr}(X^{-1}).
$$

This is a special case of the L-optimal design that we computed earlier: it is
$\phi_{\text{A}} = \phi_{K,\lambda}$ for $K = I$ and $\lambda = 0$.
Unfortunately, the group lasso reformulation we had used does not extend to the
limit case of $\lambda = 0$. So we will need another trick to get rid of the
matrix inverse.

For brevity we may minimize $-\phi_{\text{A}}(M_A(w))$ instead of maximizing
$\phi_{\text{A}}(M_A(w))$, which is equivalent up to the sign of the objective
value. We then arrive at the following problem formulation:

$$
\begin{align*}
    \text{minimize} ~&~ \operatorname{tr}({M_A(w)}^{-1}) \tag{6 | A-optimal design} \\
    \text{subject to} ~&~ \mathbf{1}^T w = 1 \\
    \text{where} ~&~ w \in \mathbb{R}_{\geq 0}^n.
\end{align*}
$$

To get rid of the inverse, we can use a staple lemma in conic optimization, the
**Schur complement**:

For symmetric matrices $P$ and $R$ with $R \succ 0$, it is

$$
    \begin{bmatrix}
        P & Q^T \\
        Q & R
    \end{bmatrix} \succeq 0
    \quad\Longleftrightarrow\quad
    P - Q^T R^{-1} Q \succeq 0.
$$

You may assume that ${M_A(w)}^{-1} = \Sigma_A(w) \succ 0$.

---

<div class="alert alert-info">

   **Bonus task 2.5:**
   
   1. Use the Schur complement lemma to reformulate problem (6) as a semidefinite program.
   2. Implement the reformulated problem.
   
   <details>
   <summary style='display: list-item'>PICOS syntax</summary>

   | on paper | in picos |
   | --- | --- |
   | $X \in \mathbb{S}^d$ | `X = pc.SymmetricVariable("X", d)` |
   | $\operatorname{tr}(A)$ | `pc.trace(A)` or `A.tr` |
   | $$\begin{bmatrix}A & B\end{bmatrix}$$ | `(A & B)` |
   | $$\begin{bmatrix}A \\ B\end{bmatrix}$$ | `(A // B)` |
   | $$\begin{bmatrix}A & B \\ C & D\end{bmatrix}$$ | `pc.block([[A, B], [C, D]])` |

   </details>

   <details>
   <summary style='display: list-item'>Hint relating the trace and matrix inequalities</summary>

   - For symmetric $A$ and $B$, it is $$A \succeq B \implies \operatorname{tr}(A) \geq
\operatorname{tr}(B).$$ Otherwise, $\operatorname{tr}(A) - \operatorname{tr}(B) =
\operatorname{tr}(A - B) < 0$ would imply that $A - B$ has a negative eigenvalue,
contradicting $A - B \succeq 0$.

   - Thus, a first step could be to introduce an auxiliary symmetric matrix variable whose trace we minimize instead, and an associated linear matrix inequality.

   </details>

</div>

In [ ]:
# TODO: Implement and solve problem (6) for M(w) defined earlier.
P = pc.Problem("A-optimality")


# Display the solution in terms of the full-dimensional data.
show_solution(A_full, w)

# Save the solution for later.
M_A = ~M

#### Comparing E- and A-optimal designs

There is another way to visualize optimal designs, which also involves
ellipsoids.

Recall the statistical motivation for maximizing the information matrix: this
amounts to minimizing the covariance matrix $\Sigma(w) = {M^{-1}(w)}$ of the
least-squares estimator $\theta_{\text{OLS}}$, assuming a linear regression
setting. The E-optimal design maximizes $\lambda_{\min}(M(w))$ and thereby
minimizes $\lambda_{\max}(\Sigma(w))$ while the A-optimal design explicitly
minimizes $\operatorname{tr}(\Sigma(w))$. For a design $w$ and a given
confidence level $\alpha$, the **$\alpha$-confidence region for the true
parameter $\theta$** is the ellipsoid

$$
    \mathcal{E} = \{\theta \mid (\theta - \theta_{\text{OLS}})^T \Sigma^{-1}(w)
    (\theta - \theta_{\text{OLS}}) \leq \beta\},
$$

where $\beta$ depends on $\alpha$ and on the data dimension $d$. Note that this
ellipsoid lives in the parameter space of the linear regression model — unlike
the ellipsoid from Task 2.4 that lived in the feature space.

Setting $\beta = 1$ and $\theta_{\text{OLS}} = 0$, we can compare the shape of
this ellipsoid for different designs $w$.

---

<div class="alert alert-info">

   **Bonus task 2.6:**

   1. Complete the code below to display confidence regions for different designs.
   2. What property of the confidence ellipsoid is optimized by the E- and A-optimal design, respectively?
   3. Why are the axes of the uniform design's confidence region aligned with the coordinate axes?

   <details>
   <summary style='display: list-item'>PICOS syntax</summary>

   | on paper | in picos | note |
   | --- | --- | --- |
   | $J_{m, n}$ | `pc.J(m, n)` | all-ones matrix |
   | $\mathbf{1}_n$ | `pc.J(n)` | all-ones vector |

   </details>

   <details>
   <summary style='display: list-item'>Hint on (2)</summary>

   - For $\beta = 1$ and $\theta_{\text{OLS}} = 0$, the ellipsoid $\mathcal{E}$ is the result of applying the linear transformation $\Sigma^{\frac{1}{2}}$ to the Euclidean unit ball $\mathcal{B} = \{x \mid {\lVert x \rVert}^2 = 1\}$. What do the eigenvalues of $\Sigma^{-1}$ tell you about this transformation?
   - For a symmetric matrix, the trace equals the sum of the eigenvalues.

   </details>

   <details>
   <summary style='display: list-item'>Hint on (3)</summary>

   - Recall that we used [PCA](https://en.wikipedia.org/wiki/Principal_component_analysis) for dimensionality reduction. What does this mean for the covariance matrix of the transformed data?

   </details>

</div>

In [ ]:
# TODO: Define a constant PICOS vector representing a uniform design, which uses
#       all available samples with the same small probability.
# w_uniform =

# Check that the design is a probability vector.
assert np.all(w_uniform.np >= 0)
assert np.allclose(w_uniform.sum.np, 1)

# TODO: Compute the associated information matrix.
# M_uniform =

# Plot confidence ellipsoids for both designs.
ux, uy, uz = ellipsoid(M_uniform.np)
ex, ey, ez = ellipsoid(M_E.np)
ax, ay, az = ellipsoid(M_A.np)
trace_theta_ols = go.Scatter3d(
    x=[0], y=[0], z=[0], name="least-squares estimator", mode="markers"
)
kwargs = dict(alphahull=0, opacity=0.3, showlegend=True)
trace_u = go.Mesh3d(x=ux, y=uy, z=uz, name="uniform design", **kwargs)
trace_e = go.Mesh3d(x=ex, y=ey, z=ez, name="E-optimal design", **kwargs)
trace_a = go.Mesh3d(x=ax, y=ay, z=az, name="A-optimal design", **kwargs)
go.Figure(
    data=[trace_theta_ols, trace_u, trace_e, trace_a],
    layout=dict(
        title_text="Confidence regions for the true parameter",
        margin=dict(t=50, b=0, l=0, r=0),
        height=600,
        scene=dict(
            xaxis=dict(showticklabels=False),
            yaxis=dict(showticklabels=False),
            zaxis=dict(showticklabels=False),
            aspectmode="data",
        ),
    ),
)

The confidence region for the uniform design roughly corresponds to picking data
samples to label at random. It should appear larger than the two optimal
designs, which in turn should have a slightly different shape.

(You can click on legend entries to disable the corresponding trace.)

#### Proving the Schur complement lemma

The Schur complement is among the most useful lemmas in conic optimization.
Its proof is not too difficult. Let's recover it!

For this, we let $P \in \mathbb{S}^p$ and $R \in \mathbb{S}^r$ with $R \succ 0$
and define $A \coloneqq \begin{bmatrix} P & Q^T \\ Q & R \end{bmatrix}$ for
brevity.

---

<div class="alert alert-info">

   **Extra bonus task 2.7:** Prove the Schur complement lemma by filling in the gaps:

   <details>
   <summary style='display: list-item'>Detailed instructions</summary>

   - (a) Use a definition of $A \succeq 0$.
   - (b) Partition $z$ into $u$ and $v$ and insert the definition of $A$.
     Rewrite the scalar $u^T Q^T v$ as its transpose $v^T Q u$ to simplify a
     bit.
   - (c) Copy and paste, then think about why this makes sense. :-)
   - (d) Use the following: For $u$ and $R \succ 0$ fixed, the convex quadratic
     problem $\text{minimize}~v^T R v + 2 v^T Q u$ has the unique optimum
     solution $v^* = -R^{-1} Q u$.
   - (e) Simplify.
   - (f) Confirm that the definition of positive semidefiniteness applies.

   </details>
</div>

<div hidden>

$$
\providecommand{\TODO}{}\renewcommand{\TODO}{{\color{red}{[\ldots]}}}
$$

</div>

$$
\begin{align*}
    & & A &\succeq 0 \\
    &\overset{\text{(a)}}\Longleftrightarrow &
        \forall z \in \mathbb{R}^{p + r} \colon
        \TODO &\geq 0 \\
    &\overset{\text{(b)}}\Longleftrightarrow &
        \forall (u, v) \in \mathbb{R}^p \times \mathbb{R}^r \colon
        \TODO &\geq 0 \\
    &\overset{\text{(c)}}\Longleftrightarrow & 
        \forall u \in \mathbb{R}^p \colon
        \inf_{v \in \mathbb{R}^r} \left( \TODO \right) &\geq 0 \\
    &\overset{\text{(d)}}\Longleftrightarrow &
        \forall u \in \mathbb{R}^p \colon
        \TODO &\geq 0 \\
    &\overset{\text{(e)}}\Longleftrightarrow &
        \forall u \in \mathbb{R}^p \colon
        u^T \left( \TODO \right) u &\geq 0 \\
    &\overset{\text{(f)}}\Longleftrightarrow &
        P - Q^T R^{-1} Q &\succeq 0.
\end{align*}
$$

### Additional resources

- A number of implementations for the quadratic (group) lasso problem, including
  one in PICOS, is found at https://gitlab.com/gsagnol/qlasso.
- There has been a [chain of
  research](https://www.zib.de/projects/optimal-design-experiments) on
  experimental design here at the ZIB.

## References

- Stephen Boyd and Lieven Vandenberghe. *Convex Optimization*. Cambridge
  University Press, Cambridge, 2004. (online version:
  https://web.stanford.edu/~boyd/cvxbook/bv_cvxbook.pdf)
- Jürgen Pilz. *Bayesian Estimation and Experimental Design in Linear Regression
  Models*. Wiley, New York, 1991.
- Guillaume Sagnol, 2011. Computing optimal designs of multiresponse experiments
  reduces to second-order cone programming. *Journal of Statistical Planning and
  Inference*, 141, 1684-1708.
  [doi:10.1016/j.jspi.2010.11.031](https://doi.org/10.1016/j.jspi.2010.11.031)
  (preprint: [arXiv:0912.5467](https://arxiv.org/abs/0912.5467))
- Guillaume Sagnol and Edouard Pauwels, 2019. An unexpected connection between
  Bayes A-optimal designs and the Group Lasso. *Statistical Papers* 60, 565-584.
  [doi:10.1007/s00362-018-01062-y](https://doi.org/10.1007/s00362-018-01062-y)
  (preprint: [arXiv:1809.01931](https://arxiv.org/abs/1809.01931))
- Guillaume Sagnol and Luc Pronzato, to appear. Fast screening rules for optimal
  design via quadratic lasso reformulation.